In [1]:
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import AllChem, MACCSkeys

from scipy.stats import spearmanr
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import PredefinedSplit, GridSearchCV


from torch.utils.data import DataLoader
from lightning.pytorch.loggers import WandbLogger
import pytorch_lightning as pl
import torch
import wandb

In [2]:
import sys
sys.path.append('../../../../code')

from metrics import get_lo_metrics
from mlp_model import MoleculeDataset, LoModel

Skipped loading some Tensorflow models, missing a dependency. No module named 'tensorflow'
wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.
Skipped loading modules with pytorch-geometric dependency, missing a dependency. No module named 'torch_geometric'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. cannot import name 'DMPNN' from 'deepchem.models.torch_models' (/home/simon/miniconda3/envs/lohi_benchmark/lib/python3.10/site-packages/deepchem/models/torch_models/__init__.py)
Skipped loading some Jax models, missing a dependency. No module named 'jax'


In [3]:
train = pd.read_csv('../../../../data/lo/kdr/train_1.csv', index_col=0)
test = pd.read_csv('../../../../data/lo/kdr/test_1.csv', index_col=0)

test

,smiles,value,cluster
0,Brc1ccc(-c2nc3ccc(Nc4ncnc5ccccc45)cc3[nH]2)cc1,6.419075,51
1,C=CC(=O)Nc1ccc(-c2ccc(NC(=O)Nc3ccc(F)cc3)cc2)cn1,8.047208,32
2,C=CC(=O)Nc1ccc(-c2ccc(NC(=O)Nc3cccc(C(C)C)c3)c...,8.508638,32
3,C=CC(=O)Nc1ccc(-c2ccc(NC(=O)Nc3cccc(Cl)c3)cc2)cn1,8.474955,32
4,C=CC(=O)Nc1cccc(-c2ccc(NC(=O)Nc3c(C)cccc3C)cc2)n1,6.380687,32
...,...,...,...
432,c1ccc2c(-c3cnn4cc(-c5ccc(N6CCNCC6)cc5)cnc34)cc...,6.666150,39
433,c1ccc2c(-c3cnn4cc(-c5ccc(N6CCOCC6)cc5)cnc34)cc...,5.273191,39
434,c1ccc2c(-c3cnn4cc(-c5ccc(OCCN6CCOCC6)cc5)cnc34...,5.616364,39
435,c1ccc2c(-c3nc4cc(-n5ccnc5)ccc4[nH]3)[nH]nc2c1,7.075721,45


In [4]:
train_mols = [Chem.MolFromSmiles(x) for x in train['smiles']]
train_fps = [Chem.MACCSkeys.GenMACCSKeys(x) for x in train_mols]
train_fps = np.array(train_fps, dtype=np.float32)

test_mols = [Chem.MolFromSmiles(x) for x in test['smiles']]
test_fps = [Chem.MACCSkeys.GenMACCSKeys(x) for x in test_mols]
test_fps = np.array(test_fps, dtype=np.float32)


In [5]:
train_dataset = MoleculeDataset(train_fps, np.array(train['value'], dtype=np.float32))
test_dataset = MoleculeDataset(test_fps, np.array(test['value'], dtype=np.float32))

In [6]:
params = {
    'layers': [167, 64, 64],
    'use_dropout': False,
    'dropout': 0.0,
    'lr': 0.003,
    'l2': 0.02,
    'batch_size': 32,
}

In [7]:
param_dict = {
    'layers': [
        [167, 32, 32],
        [167, 16, 16],
        [167, 32],
        [167, 8, 4],
        [167, 4]
    ],
    'use_dropout': [True],
    'dropout': [0.0, 0.0, 0.2, 0.4, 0.6],
    'lr': [0.01],
    'l2': [0.0, 0.0, 0.001, 0.005, 0.01],
    'batch_size': [32]
}

In [8]:
from sklearn.model_selection import ParameterSampler
from pytorch_lightning.callbacks import EarlyStopping

hyperparam_list = list(ParameterSampler(param_dict, n_iter=30))

best_score = 0.0
best_hyperparam = None

for i, hyperparam in enumerate(hyperparam_list):
    early_stop_callback = EarlyStopping(
        monitor='test_spearman',  # the metric to monitor
        min_delta=0.00,  # minimum change in the monitored quantity to qualify as an improvement
        patience=10,  # number of epochs with no improvement after which training will be stopped
        verbose=False,  # verbosity mode
        mode='max'  # 'min' for minimizing the metric, 'max' for maximizing
    )

    print('HYPERPARAMETER', i)
    print(hyperparam)

    train_dataloader = DataLoader(train_dataset, batch_size=hyperparam['batch_size'], shuffle=True)
    test_dataloader = DataLoader(test_dataset, batch_size=hyperparam['batch_size'], shuffle=False)

    model = LoModel(hyperparam, train, test)
    #wandb.init(project='drd2_lo_maccs')
    #wandb_logger = WandbLogger(project='drd2_lo_maccs')
    trainer = pl.Trainer(max_epochs=150, logger=None, accelerator="gpu", devices=[0], default_root_dir=None, callbacks=[early_stop_callback], num_sanity_val_steps=0)
    trainer.fit(model, train_dataloaders=train_dataloader, val_dataloaders=test_dataloader)
    score = early_stop_callback.best_score.detach().cpu().numpy()
    if score > best_score:
        best_score = score
        best_hyperparam = hyperparam

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


/home/simon/miniconda3/envs/lohi_benchmark/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:67: UserWarning: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
  warning_cache.warn(
You are using a CUDA device ('NVIDIA GeForce RTX 3070 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision


HYPERPARAMETER 0
{'use_dropout': True, 'lr': 0.01, 'layers': [167, 16, 16], 'l2': 0.0, 'dropout': 0.2, 'batch_size': 32}
Sequential(
  (0): Linear(in_features=167, out_features=16, bias=True)
  (1): ReLU()
  (2): Dropout(p=0.2, inplace=False)
  (3): Linear(in_features=16, out_features=16, bias=True)
  (4): ReLU()
  (5): Dropout(p=0.2, inplace=False)
  (6): Linear(in_features=16, out_features=1, bias=True)
)


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type       | Params
-------------------------------------
0 | model | Sequential | 3.0 K 
-------------------------------------
3.0 K     Trainable params
0         Non-trainable params
3.0 K     Total params
0.012     Total estimated model params size (MB)
/home/simon/miniconda3/envs/lohi_benchmark/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 20 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/simon/miniconda3/envs/lohi_benchmark/lib/python3.10/site-packages/pytorch_lightning/loops/fit_loop.py:280: PossibleUserWarning: The number of training batches (16) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_e

Training: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [9]:
best_score

array(0.16366953)

In [10]:
best_hyperparam 

{'use_dropout': True,
 'lr': 0.01,
 'layers': [167, 32, 32],
 'l2': 0.0,
 'dropout': 0.0,
 'batch_size': 32}

In [11]:
best_hyperparam = {'use_dropout': True,
 'lr': 0.01,
 'layers': [167, 32, 32],
 'l2': 0.0,
 'dropout': 0.0,
 'batch_size': 32}

In [12]:
from pytorch_lightning.callbacks import EarlyStopping
early_stop_callback = EarlyStopping(
    monitor='test_spearman',  # the metric to monitor
    min_delta=0.00,  # minimum change in the monitored quantity to qualify as an improvement
    patience=10,  # number of epochs with no improvement after which training will be stopped
    verbose=False,  # verbosity mode
    mode='max'  # 'min' for minimizing the metric, 'max' for maximizing
)
train_dataloader = DataLoader(train_dataset, batch_size=best_hyperparam['batch_size'], shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=best_hyperparam['batch_size'], shuffle=False)

model = LoModel(best_hyperparam, train, test)
#wandb.init(project='drd2_lo_maccs')
#wandb_logger = WandbLogger(project='drd2_lo_maccs')
trainer = pl.Trainer(max_epochs=150, logger=None, accelerator="gpu", devices=[0], default_root_dir=None, callbacks=[early_stop_callback], num_sanity_val_steps=0)
trainer.fit(model, train_dataloaders=train_dataloader, val_dataloaders=test_dataloader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3070 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type       | Params
-------------------------------------
0 | model | Sequential | 6.5 K 
-------------------------------------
6.5 K     Trainable params
0         Non-trainable params
6.5 K     Total params
0.026     Total estimated model params size (MB)


Sequential(
  (0): Linear(in_features=167, out_features=32, bias=True)
  (1): ReLU()
  (2): Dropout(p=0.0, inplace=False)
  (3): Linear(in_features=32, out_features=32, bias=True)
  (4): ReLU()
  (5): Dropout(p=0.0, inplace=False)
  (6): Linear(in_features=32, out_features=1, bias=True)
)


/home/simon/miniconda3/envs/lohi_benchmark/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 20 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/simon/miniconda3/envs/lohi_benchmark/lib/python3.10/site-packages/pytorch_lightning/loops/fit_loop.py:280: PossibleUserWarning: The number of training batches (16) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(
/home/simon/miniconda3/envs/lohi_benchmark/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottlen

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [13]:
all_preds = np.squeeze(torch.cat(all_preds).cpu().numpy())
all_true = np.squeeze(torch.cat(all_true).cpu().numpy())

NameError: name 'all_preds' is not defined

In [60]:
get_lo_metrics(test, all_preds)

{'r2': -4.94154284829526,
 'spearman': 0.13568143137392116,
 'mae': 0.6690745168882365}

# Final Prediction

In [13]:
def fit_predict(train, test):
    train_mols = [Chem.MolFromSmiles(x) for x in train['smiles']]
    train_fps = [Chem.MACCSkeys.GenMACCSKeys(x) for x in train_mols]
    train_fps = np.array(train_fps, dtype=np.float32)

    test_mols = [Chem.MolFromSmiles(x) for x in test['smiles']]
    test_fps = [Chem.MACCSkeys.GenMACCSKeys(x) for x in test_mols]
    test_fps = np.array(test_fps, dtype=np.float32)

    train_dataset = MoleculeDataset(train_fps, np.array(train['value'], dtype=np.float32))
    test_dataset = MoleculeDataset(test_fps, np.array(test['value'], dtype=np.float32))

    train_dataloader = DataLoader(train_dataset, batch_size=best_hyperparam['batch_size'], shuffle=True)
    test_dataloader = DataLoader(test_dataset, batch_size=best_hyperparam['batch_size'], shuffle=False)

    model = LoModel(best_hyperparam, train, test)
    trainer = pl.Trainer(max_epochs=1, accelerator="gpu", devices=[0], default_root_dir=None, num_sanity_val_steps=0)
    trainer.fit(model, train_dataloaders=train_dataloader)

    train_result = train.copy()
    train_predict_dataloader = DataLoader(train_dataset, batch_size=best_hyperparam['batch_size'], shuffle=False)
    all_preds = []
    with torch.no_grad():
        for _, batch in enumerate(train_predict_dataloader):
            X, _ = batch
            y_pred = model(X)
            all_preds.append(y_pred)
    all_preds = np.squeeze(torch.cat(all_preds).cpu().numpy())
    train_result['preds'] = all_preds

    test_result = test.copy()
    all_preds = []
    with torch.no_grad():
        for _, batch in enumerate(test_dataloader):
            X, _ = batch
            y_pred = model(X)
            all_preds.append(y_pred)
    all_preds = np.squeeze(torch.cat(all_preds).cpu().numpy())
    test_result['preds'] = all_preds

    return train_result, test_result

In [14]:
for i in [1, 2, 3]:
    train = pd.read_csv(f'../../../../data/lo/kdr/train_{i}.csv')
    test = pd.read_csv(f'../../../../data/lo/kdr/test_{i}.csv')

    train_preds, test_preds = fit_predict(train, test)
    train_preds.to_csv(f'../../../../predictions/lo/kdr/mlp_maccs/train_{i}.csv')
    test_preds.to_csv(f'../../../../predictions/lo/kdr/mlp_maccs/test_{i}.csv')


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/simon/miniconda3/envs/lohi_benchmark/lib/python3.10/site-packages/pytorch_lightning/trainer/configuration_validator.py:72: PossibleUserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn(
You are using a CUDA device ('NVIDIA GeForce RTX 3070 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type       | Params
-------------------------------------
0 | model | Sequential | 6.5 K 
-------------------------------------
6.5 K     Trainable params
0         Non-

Sequential(
  (0): Linear(in_features=167, out_features=32, bias=True)
  (1): ReLU()
  (2): Dropout(p=0.0, inplace=False)
  (3): Linear(in_features=32, out_features=32, bias=True)
  (4): ReLU()
  (5): Dropout(p=0.0, inplace=False)
  (6): Linear(in_features=32, out_features=1, bias=True)
)


/home/simon/miniconda3/envs/lohi_benchmark/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 20 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/simon/miniconda3/envs/lohi_benchmark/lib/python3.10/site-packages/pytorch_lightning/loops/fit_loop.py:280: PossibleUserWarning: The number of training batches (16) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=1` reached.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/simon/miniconda3/envs/lohi_benchmark/lib/python3.10/site-packages/pytorch_lightning/trainer/configuration_validator.py:72: PossibleUserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn(
You are using a CUDA device ('NVIDIA GeForce RTX 3070 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type       | Params
-------------------------------------
0 | model | Sequential | 6.5 K 
--------------------------------

...

`Trainer.fit` stopped: `max_epochs=1` reached.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/simon/miniconda3/envs/lohi_benchmark/lib/python3.10/site-packages/pytorch_lightning/trainer/configuration_validator.py:72: PossibleUserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn(
You are using a CUDA device ('NVIDIA GeForce RTX 3070 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type       | Params
-------------------------------------
0 | model | Sequential | 6.5 K 
--------------------------------

Sequential(
  (0): Linear(in_features=167, out_features=32, bias=True)
  (1): ReLU()
  (2): Dropout(p=0.0, inplace=False)
  (3): Linear(in_features=32, out_features=32, bias=True)
  (4): ReLU()
  (5): Dropout(p=0.0, inplace=False)
  (6): Linear(in_features=32, out_features=1, bias=True)
)


/home/simon/miniconda3/envs/lohi_benchmark/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 20 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/simon/miniconda3/envs/lohi_benchmark/lib/python3.10/site-packages/pytorch_lightning/loops/fit_loop.py:280: PossibleUserWarning: The number of training batches (16) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=1` reached.
